In [6]:
import pickle as pkl
from tsp_qaoa import TSP_QAOA

In [7]:
with open("./data/coordinates.pkl", "rb") as data:
    coordinates = pkl.load(data)

In [9]:
nlayer = 1
niter = 1

In [ ]:
results_n2 = []
for i in range(len(coordinates)):
    print(f"Sample {i+1} started:")
    tsp_qaoa = TSP_QAOA("n2", coordinates[i])
    tsp_qaoa.generate_coeffs()
    tsp_qaoa.generate_hamiltonians()
    tsp_qaoa.optimize_objective(nlayer=nlayer, niter=niter)
    tsp_qaoa.results_from_params()
    results_n2.append(tsp_qaoa)

In [ ]:
results_nlogn = []
for i in range(len(coordinates)):
    print(f"Sample {i+1} started:")
    tsp_qaoa = TSP_QAOA("nlogn", coordinates[i])
    tsp_qaoa.generate_coeffs()
    tsp_qaoa.generate_hamiltonians()
    tsp_qaoa.optimize_objective(nlayer=nlayer, niter=niter)
    tsp_qaoa.results_from_params()
    results_nlogn.append(tsp_qaoa)

In [ ]:
aggregate_result = {}
for metric in ["ar", "tp", "rn"]:
        aggregate_result[metric + "_n2"] = np.empty((len(coordinates), nlayer))
        aggregate_result[metric + "_nlogn"] = np.empty((len(coordinates), nlayer))

aggregate_serr = {}
aggregate_mean = {}
for k, v in aggregate_result.items():
    aggregate_serr[k] = v.std(axis=0) / np.sqrt(len(coordinates)) * 1.96  # 0.95
    aggregate_mean[k] = v.mean(axis=0)

In [ ]:
def subplot(ax, d1, d2, yerr1, yerr2, ylabel):
    ax.errorbar(np.arange(1,6), d2, yerr2, label="$n^2$ encoding", linestyle="dashed", c='royalblue',
                solid_capstyle='projecting', capsize=5, marker="D", markersize=5)
    ax.errorbar(np.arange(1,6), d1, yerr1, label="$n log n$ encoding", linestyle="dotted", c='green',
                solid_capstyle='projecting', capsize=5, marker="s", markersize=5)
    ax.legend()
    ax.set_xlabel("Number of Layers")
    ax.set_ylabel(ylabel)
    ax.set_xticks(np.arange(1,6))
    ax.set_xlim([0.75, 5.8])
    ax.grid(True, linestyle='dotted', alpha=0.5)

In [ ]:
fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=(15, 4), dpi=100)
plt.subplots_adjust(wspace=0.22)

subplot(ax1, aggregate_mean["ar_nlogn"], aggregate_mean["ar_n2"],
        aggregate_serr["ar_nlogn"], aggregate_serr["ar_n2"], "AR")
ax1.annotate(round(aggregate_mean["ar_n2"][-1], 1), [5.2, aggregate_mean["ar_n2"][-1]-0.2])
ax1.annotate(round(aggregate_mean["ar_nlogn"][-1], 1), [5.2, aggregate_mean["ar_nlogn"][-1]-0.2])
ax1.axhline(1, linestyle='--', c='grey', )
ax1.set_yticks(np.arange(1, 15, 2))


subplot(ax2, aggregate_mean["tp_nlogn"]*100, aggregate_mean["tp_n2"]*100,
        aggregate_serr["tp_nlogn"]*100, aggregate_serr["tp_n2"]*100, "True Percentage")
ax2.annotate(round(aggregate_mean["tp_n2"][-1]*100, 1), [5.2, aggregate_mean["tp_n2"][-1]*100-0.2])
ax2.annotate(round(aggregate_mean["tp_nlogn"][-1]*100, 1), [5.2, aggregate_mean["tp_nlogn"][-1]*100-0.2])
ax2.set_yticks(np.arange(0, 21, 2))

subplot(ax3, aggregate_mean["rn_nlogn"], aggregate_mean["rn_n2"],
        aggregate_serr["rn_nlogn"], aggregate_serr["rn_n2"], "Rank")
ax3.annotate(round(aggregate_mean["rn_n2"][-1], 1), [5.2, aggregate_mean["rn_n2"][-1]-0.2])
ax3.annotate(round(aggregate_mean["rn_nlogn"][-1], 1), [5.2, aggregate_mean["rn_nlogn"][-1]-0.2])
ax3.set_yscale('log')
ax3.axhline(1, linestyle='--', c='grey', )

plt.show()